# Importando Bibliotecas

In [ ]:
import os
import pickle
import gc
import sys
import argparse
import numpy as np

import torch
from utils import *
import pandas as pd

from minisom import MiniSom
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

from som_dagmm.model import DAGMM, SOM_DAGMM
from som_dagmm.compression_network import CompressionNetwork
from som_dagmm.estimation_network import EstimationNetwork
from som_dagmm.gmm import GMM, Mixture

from sklearn.model_selection import train_test_split

from SOM import som_train, som_pred

Definição dos parâmetros para treinamento do modelo

In [ ]:
epochs = 50
batch_size = 1024
save_path = 'saves/save/epoch_' # Caminho para salvar o modelo
dataset = 'IDS2018'             # Dataset a ser treinado e validado
features = 'numerical'
embed = 'label_encode'

# Carregando o Dataset

Durante o carregamento dos dados, já há o split entre os dados de ataque e normais, a fim de facilitar o trabalho a depender do teste que está sendo realizado.

In [ ]:
if dataset == 'IDS2018':
    data = load_data('data/NEW-CSE-CIC-IDS2018/new-500k.csv')

    dataB = data[(data['Label'] ==  "Benign")] #Pegando somente os benignos
    dataM = data[(data['Label'] !=  "Benign")]
    categorical_cols = []
    YB = get_labels(dataB, dataset)
    YM = get_labels(dataM, dataset)

if dataset == 'IDS2019':
    data = load_data('data/CSE-CIC-IDS2018/CSE-CIC-IDS2019_23.csv')

    # Retirando dados que não auxiliam no treinamento
    data = data.drop(['Flow ID', 'Source IP', 'Source Port', 'Destination IP','Destination Port', 'Protocol', 'Timestamp', 'SimillarHTTP'], axis='columns')
    dataB = data[(data['Label'] ==  "BENIGN")] #Pegando somente os benignos
    dataM = data[(data['Label'] !=  "BENIGN")]
    categorical_cols = []

    YB = get_labels(dataB, dataset)
    YM = get_labels(dataM, dataset)

Renomeia algumas variáveis para facilitar o reuso

In [ ]:
data = dataM
Y = YM

# Processamento dos Dados

Processamento dos dados de entradak, o primeiro conjunto de if's tem como objetivo escolher entre trabalhar com as features categoricas ou não do dataset. Já o segundo conjunto é realizar a conversão dessas features categoricas para numéricas

In [ ]:
#Select features
if features == "categorical":
    data = data[categorical_cols]
    dataB = dataB[categorical_cols]
if features == "numerical":
    data = remove_cols(data, categorical_cols)
    dataB = remove_cols(dataB, categorical_cols)

#encode categorical variables
if embed == 'one_hot':
    data = one_hot_encoding(data, categorical_cols)
    dataB = one_hot_encoding(dataB, categorical_cols)
if embed == 'label_encode':
    data = label_encoding(data, categorical_cols)
    dataB = label_encoding(dataB, categorical_cols)

In [ ]:
print(data.shape[0])
print(len(Y))

No código abaixo, iremos tratar respectivamente
- A existência de valores infinitos, os quais são convertidos para nan
- A troca de valopes nan por zero, dentro do fill_na
- Por fim a normalização das colunas (Utilizando o código realizado pelo artigo, para isso ele utiliza um MinMaxScaler)

In [ ]:
# Remove columns with NA values
data.replace([np.inf, -np.inf], np.nan, inplace=True)
dataB.replace([np.inf, -np.inf], np.nan, inplace=True)

data = fill_na(data)
dataB = fill_na(dataB)

# normalize data
data = normalize_cols(data)
dataB = normalize_cols(dataB)

# Separando os Dados em Treino e Validação

Abaixo é realizado o split dos dados conforme explicitado no artigo

In [ ]:
#test and train split
train_data, test_data, Y_train, Y_test = split_data(data, Y, 0.5)

In [ ]:
test_data = pd.concat([test_data, dataB], ignore_index=True)

In [ ]:
Y_test = np.concatenate([Y_test, YB])

In [ ]:
train_data = train_data.drop(train_data.index[-1])
# Y_test = Y_test[:-1]

print(len(train_data))
print(len(Y_train))
print(len(test_data))
print(len(Y_test))

Conversão dos dados para tensor Torch, a fim de que eles possam ser utilizados no treinamento e validação do modelo

In [ ]:
#Convert to torch tensors
dataX = torch.tensor(data.values.astype(np.float32))
train_dataT = torch.tensor(train_data.values.astype(np.float32))
test_dataT = torch.tensor(test_data.values.astype(np.float32))

#Convert tensor to TensorDataset class.
dataset = TensorDataset(train_dataT)

#TrainLoader
dataloader = DataLoader(train_dataT, batch_size= batch_size, shuffle=True)

# Treinamento e Validação do SOM-DAGMM

Conforme explicitado no artigo, primeiramente há o treinamento do SOM e após esse treinamento sua saída é colocada junto ao DAGMM para realizar as predições

In [ ]:
pretrained_som = som_train(data=train_dataT, x=10, y=10, sigma=1, learning_rate=0.8, iters=10000, neighborhood_function= 'gaussian')

Abaixo há o treinamento do DAGMM, bem como a delcaração dos modelos

In [ ]:
compression = CompressionNetwork(dataX.shape[1])
estimation = EstimationNetwork()
gmm = GMM(2,6)
mix = Mixture(6)
dagmm = DAGMM(compression, estimation, gmm)
net = SOM_DAGMM(dagmm, pretrained_som)
optimizer =  optim.Adam(net.parameters(), lr=0.0001)
for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch + 1))
    running_loss = 0
    for i, data in enumerate(dataloader):
        out = net(data)
        optimizer.zero_grad()

        #Calculo do loss
        L_loss = compression.reconstruction_loss(data[0])
        G_loss = mix.gmm_loss(out=out, L1=0.1, L2=0.005)

        loss = L_loss + G_loss
        

        #Retropropagação da loss
        loss.backward()
        optimizer.step()

        if torch.isfinite(loss):
            running_loss += loss.item()
            print(f"LOSS: {loss.item()} - L: {L_loss} - G: {G_loss}")
        else:
            print(f"ERROR: {loss.item()} - L: {L_loss} - G: {G_loss}")


    if (epoch+1) % 5 == 0: #Salva o modelo a cada 5 épocas
        path = save_path + str(epoch+1)
        torch.save(net, path)

        # Avalia o resultado do treinamento provisório
        net.eval()
        out_ = net(test_dataT)
        threshold = np.percentile(out_, 20)
        pred = (out_ > threshold).numpy().astype(int)

        # Precision, Recall, F1
        a, p, r, f, auc = get_scores(pred, Y_test)
        print("Accuracy:", a, "Precision:", p, "Recall:", r, "F1 Score:", f, "AUROC:", auc)
            
        print(running_loss)


Ultimo eval para verificar o desempenho final do modelo

In [ ]:
net.eval()
out_ = net(test_dataT)
threshold = np.percentile(out_, 20)
pred = (out_ > threshold).numpy().astype(int)

# Precision, Recall, F1
a, p, r, f, auc = get_scores(pred, Y_test)
print("Accuracy:", a, "Precision:", p, "Recall:", r, "F1 Score:", f, "AUROC:", auc)